In [1]:
import os
import datetime
import numpy as np
import pandas as pd

import QuantLib as ql

In [2]:
from quant_lib.swap_curve import get_quote, swap_curve

In [3]:
class IRS():
    def __init__(self, today, pricing_date, maturity_date, irs_rate, notional, position, spread=0.0):
        
        # initial setup
        self.date = today
        self.curve = self.CURVE(self.date)

        self.pricing_date = ql.Date(pricing_date.day, pricing_date.month, pricing_date.year)
        self.maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year)

        self.calendar = ql.UnitedStates()
        self.convention = ql.ModifiedPreceding
        self.day_counter = ql.Actual360()

        self.fixed_tenor = ql.Period(1, ql.Years)
        self.float_tenor = ql.Period(3, ql.Months)

        self.irs_rate = irs_rate
        self.notional = notional
        if position == 'long':
            self.position = ql.VanillaSwap.Payer
        else:
            self.position = ql.VanillaSwap.Receiver

        self.spread = spread

        # pricing result
        self.npv = self.PRICING(self.curve)
        self.delta = self.DELTA()
        self.theta = self.THETA()
    
    def CURVE(self, date):
        return swap_curve(date, get_quote(date))
    

    def PRICING(self, curve):

        #yield term structure
        curve_handle = ql.YieldTermStructureHandle(curve)

        # USD 3M Libor
        float_index = ql.USDLibor(ql.Period(3, ql.Months), curve_handle)

        # Fixed Schedule
        fixedSchedule= ql.Schedule(self.pricing_date, # effectiveDate
                                    self.maturity_date, # terminationDate
                                    self.fixed_tenor, # tenor
                                    self.calendar, # calendar
                                    self.convention, # convention
                                    self.convention, # terminationDateConvention
                                    ql.DateGeneration.Backward, # rule
                                    False  # endOfMonth
                    )

        # Fixed Schedule
        floatingSchedule= ql.Schedule(self.pricing_date, # effectiveDate
                                    self.maturity_date, # terminationDate
                                    self.float_tenor, # tenor
                                    self.calendar, # calendar
                                    self.convention, # convention
                                    self.convention, # terminationDateConvention
                                    ql.DateGeneration.Backward, # rule
                                    False # endOfMonth
                    )


        # Interest Rate Swap
        irs = ql.VanillaSwap(self.position,
                            self.notional,
                            fixedSchedule,
                            self.irs_rate,
                            self.day_counter,
                            floatingSchedule,
                            float_index,
                            self.spread,
                            self.day_counter
                )

        # pricing engine
        swapEngine = ql.DiscountingSwapEngine(curve_handle)
        irs.setPricingEngine(swapEngine)

        # IRS pricing
        npv = irs.NPV()
        
        return npv

    
    def DELTA(self):
       # delta is change in values if 1bp of interest rate curve changes
       # in here we use KRD (Key Rate Delta) not DV01
       # KRD means how each tenor changes
        curve_handle = ql.YieldTermStructureHandle(self.curve)

        basis_point = 0.0001

        # irs price when 1bp up
        up_curve = ql.ZeroSpreadedTermStructure(
                                                curve_handle,
                                                ql.QuoteHandle(ql.SimpleQuote(basis_point))
                                                )
        up_irs = self.PRICING(up_curve)

        down_curve = ql.ZeroSpreadedTermStructure(
                                                curve_handle,
                                                ql.QuoteHandle(ql.SimpleQuote(-basis_point))
                                                )
        down_irs = self.PRICING(down_curve)

        #DV01 
        delta = (up_irs - down_irs)/2
        
        return delta

    def THETA(self):
        # theta is change in value if one unit time passes.
        # in here, unit time is 1 day
        # since derivative product have time value, time to maturity is major variable in pricing derivatives
        price_t0 = self.PRICING(self.CURVE(self.date))
        price_t1 = self.PRICING(self.CURVE(self.date + datetime.timedelta(days=1)))

        return price_t1 - price_t0

In [4]:
todays_date = datetime.date(2020, 10, 9)
pricing_date = datetime.date(2021, 1, 9)
maturity_date = datetime.date(2021, 4, 9)

In [5]:
position = 'long'
irs_rate = 0.00218
notional = 1000000

irs = IRS(
    today=todays_date,
    pricing_date=pricing_date,
    maturity_date=maturity_date,
    irs_rate=irs_rate,
    notional=notional,
    position=position,
    spread=0.0
    )

In [6]:
print("price of IRS = {}".format(irs.npv))
print("delta of IRS = {}".format(irs.delta))
print("theta of IRS = {}".format(irs.theta))

price of IRS = 16.134813731714075
delta of IRS = 25.262571390637277
theta of IRS = -2.3642255974538102
